In [1]:
#Make necessary imports

import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
import FeatureEngineering as fe

/Users/kamielfokkink/opt/anaconda3/envs/Databases/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#Load in data

train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test-full.csv")

In [4]:
fe_train_data = fe.featureEngineer(train_data, soiltype=True, hydrology=True, cyclical=True, hillshade=True)
fe_test_data = fe.featureEngineer(test_data, soiltype=True, hydrology=True, cyclical=True, hillshade=True)

In [3]:
pca_train = fe.PCAtransform(train_data, 40)
pca_test = fe.PCAtransform(test_data, 40)

In [7]:
classifier = RandomForestClassifier()
#classifier = XGBClassifier()
#classifier = HistGradientBoostingClassifier()

pipe = make_pipeline(
    classifier
)

In [8]:
#pipe.fit(pca_train, train_data['Cover_Type'])
pipe.fit(fe_train_data.drop(columns=['Cover_Type']), fe_train_data['Cover_Type'])

Pipeline(steps=[('randomforestclassifier', RandomForestClassifier())])

In [6]:
def predict(pipe, data, outfile):
    ids = data['Id']
    prediction = pipe.predict(data)
    output = pd.DataFrame(prediction, index=ids, columns=['Cover_Type'])
    output.to_csv("submissions/" + outfile + ".csv", index=True)

In [11]:
predict(pipe, fe_test_data, 'RandomForestClassifier2')

### Neural Network Models

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2

In [ ]:
def make_NN_model(layer_sizes, dropout):
    model = Sequential()
    model.add(Dense(12, input_dim=55, kernel_regularizer=l2(0.05), activation='relu'))
    if dropout > 0:
        model.add(Dropout(dropout))
    for size in layer_sizes[1:]:
        model.add(Dense(size, activation='relu'))
        if dropout > 0:
            model.add(Dropout(dropout))
        
    model.add(Dense(7, kernel_regularizer=l2(0.05), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

model = make_NN_model([20,10])
model.fit(train.drop(columns=['Cover_Type']), pd.get_dummies(train['Cover_Type']), batch_size=64, epochs=40)

In [ ]:
pred = model.predict(test)
prediction = []
for row in pred[:,]:
    prediction.append(np.argmax(row)+1)

ids = test_data['Id']
output = pd.DataFrame(prediction, index=ids, columns=['Cover_Type'])

output.to_csv('NeuralNetworkClassifier4.csv') 